# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [1]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [2]:
url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Munic%C3%ADpio'    
r = requests.get(url)
r.status_code
data_json = r.json()
data = pd.DataFrame(data_json['items'])

In [4]:
# 2) Seu código aqui
status_freq = data.status.value_counts()
mais_freq = status_freq[:3]
mais_freq

Deferido        92
Arquivado       53
Regularizado    25
Name: status, dtype: int64

In [5]:
# 3) Seu código aqui
selecao = data.data_status
ano_status = selecao.str.split('/' ,expand=True)
ano_status = ano_status.loc[:, 2]
ano_status

0      2020
1      2018
2      2019
3      2003
4      2007
       ... 
230    2014
231    2015
232    2002
233    2013
234    2011
Name: 2, Length: 235, dtype: object

In [6]:
# 4) Seu código aqui

ano = pd.Series(ano_status)
ano.value_counts()

2008    37
2007    29
2014    20
2010    19
2019    17
2020    15
2013    15
2022    14
2012    13
2011    10
2015     9
2009     9
2021     5
2017     5
2018     4
2016     4
2002     4
2006     4
2003     1
2004     1
Name: 2, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [7]:
#1) Seu código aqui
# url 1 'http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=SP&tipo_interessado=Estado'
# url 2 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Munic%C3%ADpio'

def consulta (uf, tipo):
    if tipo == 'estado' :
        url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=xxx&tipo_interessado=Estado'
        novo_url = url.replace('xxx', uf)
        r = requests.get(novo_url)
        r.status_code
        data_json = r.json()
        df = pd.DataFrame(data_json['items'])
    else :
        url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=xxx&tipo_interessado=Munic%C3%ADpio'
        novo_url = url.replace('xxx', uf)
        r = requests.get(novo_url)
        r.status_code
        data_json = r.json()
        df = pd.DataFrame(data_json['items'])
    return df

consulta('BA', 'municipio')

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,41600,Município,Catu,2907509,BA,PVL02.009261/2019-93,Arquivado a pedido,17944.100385/2020-17,2020-09-30T13:45:09Z,Operação contratual interna (com garantia da U...,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,8000000.00,0,0,15/09/2022
1,12461,Município,Jucuruçu,2918456,BA,None,Arquivado por decurso de prazo,17944.001691/2010-91,2011-10-13T03:00:00Z,Operação contratual interna,Provias 2010,Instituição Financeira Nacional,Banco do Brasil S/A,Real,477000.00,0,0,16/01/2013
2,25098,Município,Salvador,2927408,BA,PVL02.000941/2017-80,Encaminhado à PGFN com manifestação técnica fa...,17944.000632/2017-72,2018-03-14T19:26:37Z,Operação contratual externa (com garantia da U...,Saneamento básico,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,67500000.00,1,0,02/04/2018
3,22504,Município,Itaberaba,2914703,BA,00000.000000/2029-54,Arquivado por decurso de prazo,17944.000613/2016-65,2016-04-26T03:00:00Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Agência de Fomento do Estado da Bahia S/A,Real,6500000.00,0,0,25/10/2016
4,34570,Município,América Dourada,2901155,BA,PVL02.003522/2019-61,Deferido (PVL-IF),None,None,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Agência de Fomento do Estado da Bahia S/A,Real,1000000.00,0,1,09/09/2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
872,4982,Município,Morro do Chapéu,2921708,BA,None,Regularizado,17944.000015/2012-62,2019-09-03T03:00:00Z,Operação contratual interna,Regularização de Dívida - Água e Esgoto,Empresa Estatal,Empresa Baiana de Águas e Saneamento S/A,Real,185867.35,0,0,23/09/2019
873,12614,Município,Piritiba,2924801,BA,None,Arquivado por decurso de prazo,17944.001728/2014-13,2014-11-12T02:00:00Z,Operação contratual interna,"Aquisição de máquinas, equipamentos e veículos",Instituição Financeira Nacional,Agência de Fomento do Estado da Bahia S/A,Real,130000.00,0,0,18/02/2015
874,13340,Município,Jequié,2918001,BA,None,Arquivado por decurso de prazo,17944.002037/2011-86,2012-08-31T03:00:00Z,Operação contratual interna,PAC 2 - Pró-Transporte - Pavimentação e Qualif...,Instituição Financeira Nacional,Caixa Econômica Federal,Real,21840000.00,0,0,08/10/2012
875,18496,Município,Nazaré,2922508,BA,None,Deferido,19407.000035/2003-55,2003-06-17T03:00:00Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Agência de Fomento do Estado da Bahia S/A,Real,1100000.00,0,0,24/06/2003


In [8]:
# 2) Seu código aqui
mg = consulta('MG', 'estado')
mg['status'].value_counts()
# Arquivado por decurso de prazo = 1

Encaminhado à PGFN com manifestação técnica favorável    20
Deferido                                                  7
Arquivado a pedido                                        5
Arquivado                                                 3
Arquivado pela STN                                        3
Indeferido                                                1
Arquivado por decurso de prazo                            1
Name: status, dtype: int64

In [9]:
# 3) Seu código aqui
ba = consulta('BA', 'municipio')
selecao = ba.groupby(['interessado', 'status'])[('status')].count().sort_values(ascending=False)
selecao.iloc[:1]

interessado             status  
Luís Eduardo Magalhães  Deferido    14
Name: status, dtype: int64

In [10]:
# 4) Seu código aqui 
estado_bahia = consulta('BA', 'estado')
estado_bahia.to_csv('Solicitações do estado da Bahia.csv')